In [8]:
import pandas as pd
import string
import time
data = pd.read_csv("Womens_Clothing_E-Commerce_Reviews.csv")

In [9]:
data.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [10]:
print(len(data))
data.drop_duplicates(subset=['Rating','Review Text'],keep='first',inplace=True)
print(len(data))

23486
22639


In [11]:
def clean_text(text ): 
    delete_dict = {sp_character: ' ' for sp_character in string.punctuation} 
    delete_dict[' '] = ' ' 
    table = str.maketrans(delete_dict)
    text1 = text.translate(table)
    #print('cleaned:'+text1)
    textArr= text1.split()
    text2 = ' '.join([w for w in textArr if ( not w.isdigit() and  ( not w.isdigit() and len(w)>2))]) 
    
    return text2.lower().split(' ')

In [12]:
df1 = data.sample(n=20000)
print(len(df1))
df1.dropna(axis = 0, how ='any',inplace=True) 
df1['Review Text']= df1['Review Text'].apply(clean_text)
print(len(df1))

20000
17335


In [13]:
sentences = df1['Review Text'].tolist()

In [14]:
print(len(sentences))
print(sentences[1])
print(sentences[17331-1])

17335
['and', 'this', 'top', 'was', 'too', 'long', 'for', 'pretty', 'but', 'expensive', 'for', 'what']
['bought', 'this', 'dress', 'store', 'and', 'suits', 'the', 'body', 'better', 'than', 'the', 'picture', 'the', 'pattern', 'exquisite', 'and', 'beautiful', 'receive', 'compliments', 'every', 'time', 'wear', 'made', 'silk', 'which', 'preferred', 'alternative', 'polyester', 'but', 'other', 'reviewers', 'mentioned', 'needs', 'slip', 'and', 'for', 'expensive', 'dress', 'should', 'come', 'with', 'one', 'should', 'fully', 'lined', 'only', 'lined', 'top', 'the', 'peep', 'hole', 'deep', 'but', 'body', 'doesn', 'gap', 'much', 'lbs', 'keeper', 'for', 'for', 'its', 'beauty']


In [15]:
import gensim

In [16]:
from gensim.models.callbacks import CallbackAny2Vec
from gensim.models import Word2Vec

# init callback class
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        elif self.epoch % 100 == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        
        
        self.epoch += 1
        self.loss_previous_step = loss

In [17]:
# init word2vec class
w2v_model = Word2Vec(size = 200,
                     window = 15,
                     min_count = 2,
                     workers = 20,
                     sg = 1,
                     negative = 5,
                     sample = 1e-5)
# build vovab


w2v_model.build_vocab(sentences)

  
# train the w2v model
start = time.time()
w2v_model.train(sentences, 
                total_examples=w2v_model.corpus_count, 
                epochs=1001, 
                report_delay=1,
                compute_loss = True, # set compute_loss = True
                callbacks=[callback()]) # add the callback class
end = time.time()


print("elapsed time in seconds :"+ str(end - start))
# save the word2vec model
w2v_model.save('word2vec_women_dress.model')

Loss after epoch 0: 114128.40625
Loss after epoch 100: 63020.0
Loss after epoch 200: 80626.0
Loss after epoch 300: 54402.0
Loss after epoch 400: 69120.0
Loss after epoch 500: 56368.0
Loss after epoch 600: 42432.0
Loss after epoch 700: 42984.0
Loss after epoch 800: 44672.0
Loss after epoch 900: 41684.0
Loss after epoch 1000: 40736.0
elapsed time in seconds :265.99909806251526


In [18]:
reloaded_w2v_model = Word2Vec.load('word2vec_women_dress.model')
words = list(reloaded_w2v_model.wv.vocab)
print('Vocab size: '+str(len(words)))
w1 = 'traditional'
print("Top 3 words similar to traditional:",\
      reloaded_w2v_model.wv.most_similar(positive = w1,topn =3))
w1 = 'party'
print("Top 3 words similar to party:",\
      reloaded_w2v_model.wv.most_similar(positive = w1,topn =3))
print("Similarity between skirt and shirt:"+\
      str(reloaded_w2v_model.wv.similarity(w1="skirt",w2="shirt")))
print("Similarity between belt and waist:"+\
      str(reloaded_w2v_model.wv.similarity(w1="belt",w2="waist")))

Vocab size: 7223
Top 3 words similar to traditional: [('thickest', 0.3836168348789215), ('wireless', 0.3767821788787842), ('flecked', 0.3625113368034363)]
Top 3 words similar to party: [('holiday', 0.541772723197937), ('wore', 0.39284107089042664), ('special', 0.33264613151550293)]
Similarity between skirt and shirt:0.34564477
Similarity between belt and waist:0.43595392
